In [1]:
%matplotlib widget
import os
import shutil
import getpass
import subprocess
import numpy as np

from newton_tools import create_grid, compute_fractals
from acme import esi_cluster_setup, ParallelMap

In [2]:
# Beispiel 1a
f = lambda x,c: x**3 - 2*x + c
df = lambda x: 3*x**2 - 2
fstr = "$x^3 - 2x + {c0:.2f}$"
xmin = -5
xmax = 5 
ymin = -5 
ymax = 5
cmin = 1.1
cmax = 3
saveto = "bsp1"

# # Beispiel 2
# f = lambda x,c: np.cosh(x) - c
# df = np.sinh
# fstr = "$cosh x - {c0:.2f}$"
# xmin = -3
# xmax = 3 
# ymin = -3 
# ymax = 3
# cmin = 1.0
# cmax = 2.0
# saveto = "bsp2"

# # Beispiel 3
# f = lambda x,c: -c*x * np.sin(x)
# df = lambda x: -np.cos(x)
# fstr = "$-{c0:.2f}x sin x$"
# xmin = -0.75
# xmax = 0.75 
# ymin = -0.75
# ymax = 0.75
# cmin = 1.5
# cmax = 2.5
# saveto = "bsp3"

Nc = 100
outfolder = os.path.join(os.path.abspath(os.path.expanduser(f"/cs/home/{getpass.getuser()}")), saveto)
shutil.rmtree(outfolder, ignore_errors=True)
os.makedirs(outfolder, exist_ok=True)
c = np.linspace(cmin, cmax, Nc)
Z = create_grid(xmin=xmin, xmax=xmax, ymin=ymin, ymax=ymax)

In [3]:
# myClient = esi_cluster_setup(n_workers=10, partition="8GBXS")
myClient = esi_cluster_setup(n_workers=40, mem_per_worker="4GB", partition="E880", n_cores=2, interactive=False)

<slurm_cluster_setup> SLURM workers ready: 40/40 	[elapsed time 00:44 | timeout at 01:00]
ACME - INFO -  Parallel computing client ready, dashboard accessible at http://10.100.33.106:8787/status


In [4]:
with ParallelMap(
    compute_fractals, 
    f=f, 
    df=df, 
    z0=Z, 
    c0=c, 
    domain=[xmin, xmax, ymin, ymax], 
    fstr=fstr, 
    idx=range(c.size), 
    outfolder=outfolder, 
    n_inputs=c.size, 
    output_dir=outfolder
    ) as pmap:
    results = pmap.compute()

ACME > ANNOUNCE <  This is ACME v. 2023.4
ACME ! WARNING ! <prepare_input> Found a single callable object in positional arguments. It will be executed just once and shared by all workers
ACME ! WARNING ! <prepare_input> Found a single callable object in positional arguments. It will be executed just once and shared by all workers
ACME - INFO -  Attaching to parallel computing client <Client: 'tcp://10.100.33.106:40435' processes=40 threads=80, memory=149.20 GiB>
ACME - INFO -  Preparing 100 parallel calls of `compute_fractals` using 40 workers
100% |██████████| 100/100 [01:35<00:00]
ACME > ANNOUNCE <  SUCCESS!
ACME - INFO -  Results have been saved to /cs/home/fuertingers/bsp1/compute_fractals.h5


In [5]:
imgs = os.path.join(outfolder, "*.png")
gif = os.path.join(outfolder, f"{saveto}.gif")
subprocess.run(f"convert -size 1280x960 -delay 0 -loop 0 $(ls {imgs} | sort -V) {gif}",
               text=True, shell=True, check=True,
               stdout=subprocess.PIPE, stderr=subprocess.STDOUT)

CompletedProcess(args='convert -size 1280x960 -delay 0 -loop 0 $(ls /cs/home/fuertingers/bsp1/*.png | sort -V) /cs/home/fuertingers/bsp1/bsp1.gif', returncode=0, stdout='')

In [6]:
shutil.copy(gif, os.path.abspath(os.path.expanduser("~")))

'/gs/home/fuertingers/bsp1.gif'